# Validación del modelo de Negación e Incertidumbre (Bert-base-uncased-finetuned-ner-negation_detection_NUBes) para hacer predicciones en historias clinicas.

*NUBes: A Corpus of Negation and Uncertainty in Spanish Clinical Texts


# **1. Instalación de Librerías**
Primero se instalan las librerías necesarias para que el script pueda usar modelos de lenguaje preentrenados. La primera instala transformers con soporte para PyTorch, y la segunda accelerate, que permite ejecutar los modelos de forma eficiente en CPU o GPU.


In [1]:
!pip install transformers[torch]
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import login
login(getpass("Introduce tu token de Hugging Face: "))

Luego, se importan las librerías necesarias para el procesamiento de texto y la inferencia del modelo. `pandas` se usa para organizar y visualizar los resultados, `transformers` permite cargar el modelo y el tokenizador preentrenado, `torch` se utiliza para manejar tensores y realizar cálculos con el modelo, `F` contiene funciones útiles como softmax, y `tqdm` se emplea para mostrar una barra de progreso durante la ejecución del script.


In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import torch.nn.functional as F
from tqdm import tqdm


# **2. Definición de etiquetas**
Se define un diccionario que asigna una etiqueta textual a cada número de clase que el modelo puede predecir. Estas etiquetas indican si una palabra está asociada a negación (NEG), incertidumbre (UNC), una afirmación no especificada (NSCO o USCO), o si no pertenece a ninguna categoría (O). Luego, se calcula el número total de etiquetas (num_labels) para usarlo al cargar el modelo.

In [5]:
### Diccionario con las etiquetas usadas en el modelo
id2label = {
    0: 'B-NEG',
    1: 'B-NSCO',
    2: 'B-UNC',
    3: 'B-USCO',
    4: 'I-NEG',
    5: 'I-NSCO',
    6: 'I-UNC',
    7: 'I-USCO',
    8: 'O',
}

num_labels = len(id2label)

# **3. Carga del modelo Preentrenado y Tokenizador**

Se carga el modelo preentrenado de detección de negación e incertidumbre desde Hugging Face, junto con su tokenizador correspondiente. Se especifican las etiquetas que el modelo debe reconocer usando los diccionarios id2label y label2id. Además, se define un tamaño de lote (batch_size) para procesar los textos en grupos durante la inferencia y se prepara una lista (all_results) para almacenar los resultados.

In [6]:
# Cargar modelo y tokenizer
# Se carga el modelo entrenado previamente
hugging_face_NER_model="JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES"

model = AutoModelForTokenClassification.from_pretrained(hugging_face_NER_model,
        num_labels = num_labels,
        id2label = id2label,
        label2id = {v: k for k, v in id2label.items()}
)

tokenizer = AutoTokenizer.from_pretrained(hugging_face_NER_model, use_fast = True)


# Usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


all_results = []
batch_size = 8

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

#**4. Lectura de archivos de texto con historias clínicas**

Este bloque de código recorre una carpeta del entorno de Google Drive que contiene historias clínicas en formato .txt. Utiliza la librería pathlib para identificar todos los archivos de texto, lee su contenido y los guarda en una lista (texts). También imprime la cantidad total de archivos leídos y muestra sus nombres, lo cual permite verificar que se haya cargado correctamente toda la información necesaria para el análisis.

In [7]:
from pathlib import Path

# Ruta de la carpeta con los archivos .txt
carpeta_txt = Path("/content/drive/MyDrive/MAESTRIA_2024/2. Salud/Tarea_2_aplicacion_IE/Notas_Cancer_Mama")

# Leer todos los archivos .txt de la carpeta
texts = []
for archivo in carpeta_txt.glob("*.txt"):
    with open(archivo, "r", encoding="utf-8") as f:
        contenido = f.read().strip()
        if contenido:  # solo agrega si no está vacío
            texts.append(contenido)
print(f"Se leyeron {len(texts)} archivos.")

# Mostrar los nombres de los archivos leídos (opcional)
archivos_leidos = list(carpeta_txt.glob("*.txt"))
for i, archivo in enumerate(archivos_leidos, start=1):
    print(f"{i}. {archivo.name}")


Se leyeron 106 archivos.
1. 160.txt
2. 124.txt
3. 158.txt
4. 100.txt
5. 36127.txt
6. 2720211.txt
7. 242186_1.txt
8. 241.txt
9. 2071255.txt
10. 499061.txt
11. 912866.txt
12. 1521.txt
13. 1525.txt
14. 100(1).txt
15. 235.txt
16. 234.txt
17. 1234567m.txt
18. 142.txt
19. 1126737.txt
20. 1099899.txt
21. 3314.txt
22. 5514.txt
23. 133.txt
24. 240.txt
25. 152.txt
26. 239.txt
27. 131.txt
28. 129.txt
29. 140.txt
30. 1365655.txt
31. 162.txt
32. 157.txt
33. 153.txt
34. 236.txt
35. 54.txt
36. 161.txt
37. 159.txt
38. 212155.txt
39. 2759.txt
40. 45812.txt
41. 126.txt
42. 132.txt
43. 141.txt
44. 237.txt
45. 5547.txt
46. 151.txt
47. 2501.txt
48. 155.txt
49. 125.txt
50. 154.txt
51. 36687.txt
52. 120.txt
53. 163.txt
54. 4512.txt
55. 101.txt
56. 569236.txt
57. 138.txt
58. 149.txt
59. 330177_1.txt
60. 143.txt
61. 7754.txt
62. 69.txt
63. 146.txt
64. 144.txt
65. 2567.txt
66. 9419.txt
67. 136.txt
68. 139.txt
69. 130m.txt
70. 134.txt
71. 123.txt
72. 128.txt
73. 121.txt
74. 150.txt
75. 127.txt
76. 3389.txt
77. 5

# **5. Tokenización de los textos**
Se aplica el tokenizador del modelo preentrenado a la lista de historias clínicas.
> La función `tokenizer()` convierte cada texto en una secuencia de tokens que el modelo puede interpretar. Se habilita el truncamiento y el padding para asegurar que todos los textos tengan la misma longitud máxima (512 tokens). Además, se solicitan los mapeos de desplazamiento (offset_mapping) para poder vincular los tokens con su posición original en el texto, lo cual es útil al momento de reconstruir las entidades detectadas.


In [8]:
# Tokenización
encodings = tokenizer(
        texts,
        truncation=True,
        padding=True,
        return_offsets_mapping=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        max_length=512,
        is_split_into_words=False
        )

# 6. **Predicciones de las etiquetas**
Aquí los textos se convierten en tokens que el modelo puede entender. Luego se ejecuta el modelo en modo evaluación (torch.no_grad()), y se obtiene la probabilidad de cada etiqueta por token. Finalmente, se escoge la etiqueta más probable para cada token.

In [9]:
input_ids = torch.tensor(encodings["input_ids"]).to(device)

attention_mask = torch.tensor(encodings["attention_mask"]).to(device)

# Predicción
with torch.no_grad():
 outputs = model(input_ids=input_ids, attention_mask=attention_mask)

logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)
probs = F.softmax(logits, dim=-1)

In [ ]:
print (predictions)


# **7. Alineación de tokens con etiquetas**
En esta etapa se reconstruyen las palabras originales a partir de los subtokens generados por el modelo BERT, ya que algunas palabras pueden dividirse durante la tokenización. Luego, se asigna a cada palabra su etiqueta correspondiente, y se calcula el puntaje de confianza (score) asociado.

In [31]:
# Alineación de tokens, etiquetas y scores para el modelo de negación/incertidumbre

aligned_results = []

for i, text in enumerate(texts):
    word_ids = encodings.word_ids(batch_index=i)
    tokens = tokenizer.convert_ids_to_tokens(encodings["input_ids"][i])

    previous_word_id = None
    aligned_words, aligned_labels, aligned_scores = [], [], []

    for j, (token, label_id, word_id) in enumerate(zip(tokens, predictions[i].tolist(), word_ids)):
        if word_id is None:
            continue

        prob = probs[i][j][label_id].item()
        token_clean = token.replace("▁", "").replace("##", "")

        if word_id != previous_word_id:
            aligned_words.append(token_clean)
            aligned_labels.append(id2label[label_id])
            aligned_scores.append(prob)
        else:
            aligned_words[-1] += token_clean
            aligned_scores[-1] = max(aligned_scores[-1], prob)

        previous_word_id = word_id

    filtered_results = [
        (word, label, score)
        for word, label, score in zip(aligned_words, aligned_labels, aligned_scores)
        if label != "O"
    ]

    aligned_results.append(filtered_results)

    print(f"\nResultados alineados para el texto {i+1}:")
    #print("Texto:", text)
    print("Palabras:", aligned_words)
    print("Labels:  ", aligned_labels)
    print("Scores:  ", aligned_scores)



Resultados alineados para la oración 1:
Palabras: ['carcinoma', 'ductal', 'infiltrante', 't2n0m0', ',', 'receptores', 'hormonales', 'positivos', ',', 'her2', 'negativo', '.', 'mujer', 'premenopausica', 'tratada', 'con', 'mastectomia', 'en', 'julio', 'del', '2009', '.', 'en', 'tratamiento', 'con', 'quimioterapia', 'tipo', 'ac', 'por', '4', 'y', 'tamoxifeno', 'posterior', '.', 'recaida', 'local', 'en', 'el', 'lecho', 'de', 'mastectomia', 'en', 'agosto', 'del', '2010', '.', 'entre', 'los', 'dias', '21', '-', '12', '-', '2010', 'y', '07', '-', '01', '-', '2011', ',', 'se', 'procedio', 'a', 'irradiacion', 'de', 'pared', 'toracica', 'izquierda', ',', 'de', 'areas', 'ganglionares', 'supraclaviculares', ',', 'interpectorales', ',', 'infraclaviculares', 'y', 'niveles', 'ganglionares', 'axilares', '.', 'en', 'enero', 'del', '2011', 'iniica', 'bloqueo', 'hormonal', 'completo', 'con', 'zoladex', '+', 'tamoxifeno', ',', 'sustituido', 'por', 'letrozol', 'al', 'mes', 'siguiente', '.', 'metastasis', 

# **8. Unión de etiquetas B-I en una sola entidad y resultados del modelo**

Después de alinear los tokens, unimos las entidades que aparecen fragmentadas entre etiquetas B- (inicio) e I- (continuación). Esta unión permite reconstruir entidades completas como frases negadas o expresiones de incertidumbre. Además, se calcula el puntaje promedio de confianza de cada entidad compuesta, lo que ayuda a validar su detección con mayor seguridad.

In [32]:
# Unión de etiquetas B-I en una sola entidad

all_results = []

for i, sentence_results in enumerate(aligned_results):
    print("=" * 100)
    print(f"\n Texto {i+1}")
    print("Texto:", texts[i])

    print("\n **** Se unen las etiquetas B-, I- en una sola entidad **** \n")

    combined_results = []
    temp_entity, temp_label, temp_scores = "", "", []

    for word, label, score in sentence_results:
        if label.startswith("B-"):
            if temp_entity:
                combined_results.append((temp_entity, temp_label, round(sum(temp_scores) / len(temp_scores), 6)))
            temp_entity = word
            temp_label = label[2:]  # "NEG", "NSCO", "USCO", etc.
            temp_scores = [score]
        elif label.startswith("I-") and label[2:] == temp_label:
            temp_entity += " " + word
            temp_scores.append(score)
        else:
            if temp_entity:
                combined_results.append((temp_entity, temp_label, round(sum(temp_scores) / len(temp_scores), 6)))
            temp_entity, temp_label, temp_scores = "", "", []

    if temp_entity:
        combined_results.append((temp_entity, temp_label, round(sum(temp_scores) / len(temp_scores), 6)))

    for entity, label, score in combined_results:
        result = {
            "Palabra": entity,
            "Entidad": label,
            "Score": score
        }
        print(result)
        all_results.append(result)



 Oración 1
Texto: Carcinoma ductal infiltrante T2N0M0, receptores hormonales positivos, HER2 negativo.

Mujer Premenopausica tratada con Mastectomia en Julio del 2009.

En tratamiento con QUIMIOTERAPIA tipo AC por 4 y TAMOXIFENO posterior.

Recaida local en el lecho de mastectomía en Agosto del 2010.

Entre los días 21-12-2010 y 07-01-2011, se procedió a irradiación de pared torácica izquierda, de áreas ganglionares supraclaviculares, interpectorales, infraclaviculares y niveles ganglionares axilares.

En Enero del 2011 iniica bloqueo hormonal completo con ZOLADEX + Tamoxifeno, sustituido por LETROZOL al mes siguiente.

Metastasis tratada con cirugia y RADIOTERAPIA.

Inicio de 1ra línea de tratamiento quimioterápico para metástasis con Gemcitabina 2500 mg/m2 y Carboplatino AUC 2,5.

 **** Se unen las etiquetas B-, I- en una sola entidad **** 

{'Palabra': 'her2', 'Entidad': 'NSCO', 'Score': 0.569735}
{'Palabra': 'negativo', 'Entidad': 'NEG', 'Score': 0.848804}

 Oración 2
Texto: Servi

In [33]:
from collections import Counter

conteo_etiquetas = Counter([res["Entidad"] for res in all_results])
print("\n🧾 Conteo total de entidades detectadas:")
for entidad, count in conteo_etiquetas.items():
    print(f"{entidad}: {count}")



🧾 Conteo total de entidades detectadas:
NSCO: 368
NEG: 396
USCO: 39
UNC: 38


A partir de los resultados obtenidos, se puede concluir que el modelo de detección de negación e incertidumbre funciona correctamente sobre las historias clínicas analizadas. El modelo identificó de forma precisa entidades clínicas afirmadas (NSCO), negadas (NEG) e inciertas (UNC), asignando etiquetas coherentes con el contexto del texto. Además, los puntajes de confianza obtenidos fueron consistentemente altos (superiores a 0.8 en la mayoría de los casos), lo cual indica que el modelo tiene alta seguridad en sus predicciones. Esto sugiere que el modelo es adecuado para tareas de extracción de información en lenguaje clínico, y que puede integrarse como parte de una herramienta automatizada para analizar historias clínicas en el contexto del cáncer de mama.

En conclusión, el modelo parece estar sesgado hacia la identificación de marcadores explícitos de negación, sin generalizar hacia el alcance.
Esto compromete su utilidad para tareas como filtrado de síntomas negativos o extracción clínica automatizada.